# Galerkin block assembly

In [ ]:
from kl_decomposition import galerkin
import numpy as np


In [ ]:
A = galerkin.assemble_block((0.0, 1.0), coeff_b=2.0, n=3)
print(A)

## Fast assembly with Duffy mapping

In [ ]:
A_ref = galerkin.assemble_duffy(f=1e-1, degree=20, quad=1000, gx=4.0, gy=4.0)

In [ ]:
A_duf = galerkin.assemble_duffy(f=1e1, degree=20, quad=20, gx=2.0, gy=2.0)
err = np.linalg.norm(A_duf - A_ref)
print('Frobenius error vs reference:', err)

In [ ]:
A = galerkin.assemble_block((0.0, 1.0), coeff_b=100.0, n=20)
err = np.linalg.norm(A_duf - A)
print('Frobenius error vs reference:', err)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

f_val = 1e9

# Parameters to sweep
quad_points = range(10, 180, 20)
g_values = [1, 2, 3, 4, 5, 6]

results = []

# Reference matrix (fixed Duffy with gx=gy=4 and high quad)
A_ref = galerkin.assemble_duffy(f=f_val, degree=20, quad=1000, gx=4.0, gy=4.0)

# Duffy assembly tests
for gx in g_values:
    for quad in quad_points:
        repeats = min(int(100000 / (quad**2)), 20)
        start = time.perf_counter()
        for _ in range(repeats):
            A_duf = galerkin.assemble_duffy(f=f_val, degree=20, quad=quad, gx=gx, gy=gx)
        duration = (time.perf_counter() - start) / repeats
        err = np.linalg.norm(A_duf - A_ref)
        results.append((gx, gx, quad, err, duration))
        print(f"Duffy gx={gx:.1f}, quad={quad} -> err={err:.2e}, time={duration:.3f}s")


quad_points = range(10, 600, 50)
# Gauss2D tests with matching quad values
for quad in quad_points:
    repeats = max(min(int(1000000 / (quad**2)), 20),1)
    start = time.perf_counter()
    for _ in range(repeats):
        A_gauss = galerkin.assemble_gauss2d(f=f_val, degree=20, quad=quad)
    duration = (time.perf_counter() - start) / repeats
    err = np.linalg.norm(A_gauss - A_ref)
    results.append((np.nan, np.nan, quad, err, duration))
    print(f"Gauss2D quad={quad} -> err={err:.2e}, time={duration:.3f}s")

# Store results
df = pd.DataFrame(results, columns=["gx", "gy", "quad", "frobenius_error", "time_sec"])
df.to_csv("duffy_precision_test.csv", index=False)

In [ ]:
# === PLOT SECTION ===========================================================
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))

# Plot Duffy curves
for (gx, gy), group in df[df["gx"].notna()].groupby(["gx", "gy"]):
    label = f"{gx:.0f},{gy:.0f}"
    plt.plot(group["time_sec"], group["frobenius_error"],
             label=label, marker='o', linestyle='-')

# Plot Gauss2D entries (gx, gy == NaN)
gauss_group = df[df["gx"].isna()]
plt.plot(gauss_group["time_sec"], gauss_group["frobenius_error"],
         label="Gauss2D", marker='x', linestyle='--', color='black')

# Log scale on y-axis
plt.yscale("log")

plt.xlabel("Time per solve [s]")
plt.ylabel("Frobenius error")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)

# Legend above plot
plt.legend(
    title="gx,gy",
    loc="upper center",
    bbox_to_anchor=(0.5, 1.15),
    ncol=4,
    frameon=False
)

plt.tight_layout()
# plt.savefig("duffy_precision_plot.pdf")
plt.show()

In [ ]:
# === PLOT SECTION ===========================================================
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))

# Plot Duffy curves
for (gx, gy), group in df[df["gx"].notna()].groupby(["gx", "gy"]):
    label = f"{gx:.0f},{gy:.0f}"
    plt.plot(group["time_sec"], group["frobenius_error"],
             label=label, marker='o', linestyle='-')

# Plot Gauss2D entries (gx, gy == NaN)
gauss_group = df[df["gx"].isna()]
plt.plot(gauss_group["time_sec"], gauss_group["frobenius_error"],
         label="Gauss2D", marker='x', linestyle='--', color='black')

# Log scale on y-axis
plt.yscale("log")

plt.xlabel("Time per solve [s]")
plt.ylabel("Frobenius error")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)

# Legend above plot
plt.legend(
    title="gx,gy",
    loc="upper center",
    bbox_to_anchor=(0.5, 1.15),
    ncol=4,
    frameon=False
)

plt.tight_layout()
# plt.savefig("duffy_precision_plot.pdf")
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
df = pd.read_csv("duffy_precision_test.csv")

# Automatically select first 3 unique gx and gy values
gx_vals = sorted(df["gx"].unique())[:3]
gy_vals = sorted(df["gy"].unique())[:3]

# Filter the data
df = df[df["gx"].isin(gx_vals) & df["gy"].isin(gy_vals)]

# Label each combination
df["label"] = df.apply(lambda row: f"{row['gx']:.1f},{row['gy']:.1f}", axis=1)

# Assign colors to gx values
color_palette = sns.color_palette("tab10", n_colors=3)
gx_colors = dict(zip(gx_vals, color_palette))

# Assign marker and line styles to gy values
markers = ['o', 's', 'D']
linestyles = ['-', '--', ':']
gy_styles = dict(zip(gy_vals, zip(markers, linestyles)))

# Create plot
plt.figure(figsize=(9, 6))

for (gx, gy), group in df.groupby(["gx", "gy"]):
    label = f"{gx:.1f},{gy:.1f}"
    color = gx_colors[gx]
    marker, linestyle = gy_styles[gy]
    plt.plot(group["time_sec"], group["frobenius_error"],
             label=label,
             color=color,
             linestyle=linestyle,
            # marker=marker,
             markersize=5)

# plt.xscale("log")
plt.yscale("log")
plt.xlabel("Computation Time [s] (log scale)")
plt.ylabel("Frobenius Error (log scale)")
plt.title("Duffy Assembly Precision vs Time (3×3 combinations)")

# Legend above plot in 3 columns
plt.legend(title="gx,gy", loc="upper center", bbox_to_anchor=(0.5, 1.4),
           ncol=3, frameon=False)

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
df = pd.read_csv("duffy_precision_test.csv")

# Automatically select first 3 unique gx and gy values
gx_vals = sorted(df["gx"].unique())[:3]
gy_vals = sorted(df["gy"].unique())[:3]

# Filter the data
df = df[df["gx"].isin(gx_vals) & df["gy"].isin(gy_vals)]

# Label each combination
df["label"] = df.apply(lambda row: f"{row['gx']:.1f},{row['gy']:.1f}", axis=1)

# Assign colors to gx values
color_palette = sns.color_palette("tab10", n_colors=3)
gx_colors = dict(zip(gx_vals, color_palette))

# Assign marker and line styles to gy values
markers = ['o', 's', 'D']
linestyles = ['-', '--', ':']
gy_styles = dict(zip(gy_vals, zip(markers, linestyles)))

# Create plot
plt.figure(figsize=(9, 6))

for (gx, gy), group in df.groupby(["gx", "gy"]):
    label = f"{gx:.1f},{gy:.1f}"
    color = gx_colors[gx]
    marker, linestyle = gy_styles[gy]
    plt.plot(group["time_sec"], group["frobenius_error"],
             label=label,
             color=color,
             linestyle=linestyle,
             # marker=marker,
             markersize=5)

# plt.xscale("log")
plt.yscale("log")
plt.xlabel("Computation Time [s] (log scale)")
plt.ylabel("Frobenius Error (log scale)")
plt.title("Duffy Assembly Precision vs Time (3×3 combinations)")

# Legend above plot in 3 columns
plt.legend(title="gx,gy", loc="upper center", bbox_to_anchor=(0.5, 1.4),
           ncol=3, frameon=False)

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
df = pd.read_csv("duffy_precision_test.csv")

# Automatically select first 3 unique gx and gy values
gx_vals = sorted(df["gx"].unique())[:3]
gy_vals = sorted(df["gy"].unique())[:3]

# Filter the data
df = df[df["gx"].isin(gx_vals) & df["gy"].isin(gy_vals)]

# Label each combination
df["label"] = df.apply(lambda row: f"{row['gx']:.1f},{row['gy']:.1f}", axis=1)

# Assign colors to gx values
color_palette = sns.color_palette("tab10", n_colors=3)
gx_colors = dict(zip(gx_vals, color_palette))

# Assign marker and line styles to gy values
markers = ['o', 's', 'D']
linestyles = ['-', '--', ':']
gy_styles = dict(zip(gy_vals, zip(markers, linestyles)))

# Create plot
plt.figure(figsize=(9, 6))

for (gx, gy), group in df.groupby(["gx", "gy"]):
    label = f"{gx:.1f},{gy:.1f}"
    color = gx_colors[gx]
    marker, linestyle = gy_styles[gy]
    plt.plot(group["time_sec"], group["frobenius_error"],
             label=label,
             color=color,
             linestyle=linestyle,
             # marker=marker,
             markersize=5)

# plt.xscale("log")
plt.yscale("log")
plt.xlabel("Computation Time [s] (log scale)")
plt.ylabel("Frobenius Error (log scale)")
plt.title("Duffy Assembly Precision vs Time (3×3 combinations)")

# Legend above plot in 3 columns
plt.legend(title="gx,gy", loc="upper center", bbox_to_anchor=(0.5, 1.4),
           ncol=3, frameon=False)

plt.tight_layout()
plt.show()